In [1]:
import numpy as np

# Two-body model
Binary complex  
$$
\begin{bmatrix}
6 & 0 \\
5 & 1 \\
\end{bmatrix}
$$

In [9]:
x = np.array([6, 5, 4, 4, 3, 3, 2, 2, 1, 0])
A = np.stack([x, 6 - x], axis=1)
print(A)

[[6 0]
 [5 1]
 [4 2]
 [4 2]
 [3 3]
 [3 3]
 [2 4]
 [2 4]
 [1 5]
 [0 6]]


In [7]:
np.linalg.matrix_rank(A)

2

# Three-body model

In [20]:
A = np.array(
    [
        [6, 0, 12, 0, 0, 3, 0, 0],
        [5, 1, 8, 4, 0, 2, 1, 0],
        [4, 2, 5, 6, 1, 1, 2, 0],
        [4, 2, 4, 8, 0, 2, 0, 1],
        [3, 3, 3, 6, 3, 0, 3, 0],
        [3, 3, 2, 8, 2, 1, 1, 1],
        [2, 4, 1, 6, 5, 0, 2, 1],
        [2, 4, 0, 8, 4, 1, 0, 2],
        [1, 5, 0, 4, 8, 0, 1, 2],
        [0, 6, 0, 0, 12, 0, 0, 3],
    ]
)
# Assert that N_A + N_B = 6
np.testing.assert_equal(np.sum(A[:, :2], axis=1), 6 * np.ones(10))
# Assert that N^cis_AA + N^cis_AB + N^cis_BB = 12
np.testing.assert_equal(np.sum(A[:, 2:5], axis=1), 12 * np.ones(10))
# Assert that N^trans_AA + N^trans_AB + N^trans_BB = 3
np.testing.assert_equal(np.sum(A[:, 5:], axis=1), 3 * np.ones(10))

print(np.linalg.matrix_rank(A))

4


In [61]:
# Cis/Trans splitting:
cis_trans_split = (A[2, :] - A[3, :]) / 2
# Fac/Mer splitting
fac_mer_split = (A[4, :] - A[5, :]) / 2
# Assert they are the same
np.testing.assert_array_equal(cis_trans_split, fac_mer_split)
print(fac_mer_split)

[ 0.   0.   0.5 -1.   0.5 -0.5  1.  -0.5]


In [62]:
# Curvature
a = np.array([6, 0, 0, 0, 0, 0, 0, 0])
b = np.array([0, 6, 0, 0, 0, 0, 0, 0])
print((5 * a + 1 * b) / 6 - A[1, :])
print((4 * a + 2 * b) / 6 - (A[2, :] + A[3, :]) / 2)
print((3 * a + 3 * b) / 6 - (A[4, :] + A[5, :]) / 2)
print((2 * a + 4 * b) / 6 - (A[6, :] + A[7, :]) / 2)
print((1 * a + 5 * b) / 6 - A[8, :])

[ 0.  0. -8. -4.  0. -2. -1.  0.]
[ 0.   0.  -4.5 -7.  -0.5 -1.5 -1.  -0.5]
[ 0.   0.  -2.5 -7.  -2.5 -0.5 -2.  -0.5]
[ 0.   0.  -0.5 -7.  -4.5 -0.5 -1.  -1.5]
[ 0.  0.  0. -4. -8.  0. -1. -2.]


In [36]:
A[:, 0]

array([6, 5, 4, 4, 3, 3, 2, 2, 1, 0])

In [39]:
A[:, 2] + A[:, 5]

array([15, 10,  6,  6,  3,  3,  1,  1,  0,  0])